# Assignment 3 – Using the A* Algorithm
## Task 1.1
An implementation of the A* algorithm has been lifted from [here](https://github.com/jrialland/python-astar). The file of interest is [`astar/__init__.py`](https://github.com/jrialland/python-astar/blob/master/src/astar/__init__.py) where the `AStar` class is implemented.

The A* algorithm has been used in the mazesolver module, specifically in `mazesolver.MazeSolver`. The source code follows:

In [ ]:
# %load mazesolver/__init__.py
from typing import Tuple, List

import colored
from colored import stylize
import numpy as np

from astar import AStar


Node = Tuple[int, int]


class MazeSolver(AStar):
    """
    Each node is represented by its index (x,y) in the maze array
    """
    REACHABLE_NODES = ['A', 'B', '.']

    def __init__(self, maze: str) -> None:
        lines = maze.split('\n')

        # All lines need to be of the same width as the first line
        self.width = len(lines[0])

        items = [
            [c for c in line]
            for line
            in lines
            if len(line) == self.width
        ]
        self.height = len(items)

        self.maze = np.array(items)

        assert self.height, self.width == self.maze.shape

        start_index = np.where(self.maze == 'A')
        self.start = (start_index[0][0], start_index[1][0],)

        goal_index = np.where(self.maze == 'B')
        self.goal = (goal_index[0][0], goal_index[1][0],)

        self.solve_attempted = False

    def neighbors(self, node: Node):
        x, y = node
        nnodes = ((x + 1, y), (x, y + 1), (x - 1, y), (x, y - 1))
        return tuple(nnode for nnode in nnodes if self.reachable(nnode))

    def reachable(self, node: Node) -> bool:
        # Numpy allows negative indexing, so this needs to be guarded against
        if node[0] < 0 or node[1] < 0:
            return False

        try:
            return self.maze[node] in self.REACHABLE_NODES
        except IndexError:
            # Too big an index is out of bounds of the maze
            return False

    def distance_between(self, n1: Node, n2: Node) -> int:
        # Neighbouring nodes are always 1 units apart
        return 1

    def heuristic_cost_estimate(self, current: Node, goal: Node) -> int:
        return abs(goal[0] - current[0]) + abs(goal[1] - current[1])

    def is_goal_reached(self, current, goal):
        return current == self.goal

    def solve(self, method='Astar') -> List[Node]:
        self.path = self.astar(self.start, self.goal, method=method)

        # Successful self.astar() returns a generator, so it is cast into a
        # list
        if not self.path is None:
            self.path = list(self.path)
            self.success = True

            self.solved_maze = self.maze.copy()

            # Fill in the found path, except for start and goal
            for position in self.path[1:-1]:
                self.solved_maze[position] = 'O'
        else:
            self.success = False

        self.solve_attempted = True

        return self.path

    def visualize(self) -> None:
        if not self.solve_attempted:
            self.solve()

        if self.success:
            self.print_maze(self.solved_maze)
        else:
            print("Could not find solution to the maze!")
            self.print_maze(self.maze)

    @classmethod
    def print_maze(cls, maze) -> None:
        # Print top boarder of maze
        print('-' * (maze.shape[1] + 2))

        for row in range(0, maze.shape[0]):
            # Print left boarder of maze
            print('|', end='')

            for col in range(0, maze.shape[1]):
                # Print the text representation of each individual node
                print(
                    cls.node_representation(
                        maze=maze,
                        node=(row, col),
                    ),
                    end='',
                )

            # Print left boarder of maze
            print('|')

        # Print bottom boarder of maze
        print('-' * (maze.shape[1] + 2))

    @classmethod
    def node_representation(cls, maze: np.array, node: Node) -> str:
        STYLE_OF = {
            '.': colored.bg('black'),
            '#': colored.bg(239),
            '-': colored.bg('white'),
            '|': colored.bg('white'),
            'A': colored.bg(1),
            'B': colored.bg(1),
            'O': colored.bg('dark_orange_3b'),
        }
        type = maze[node]
        return stylize(type[-1], STYLE_OF[type[0]] + colored.fg('white'))

    def representation(
        self,
        with_path=True,
        with_open_closed=False,
    ) -> np.array:
        """
        Returns a matrix which when printed elemntwise shows a nice visual
        representation of the maze.
        """
        rows = self.height + 2
        cols = self.width + 2

        # Preallocate varstring array
        items = rows * cols
        self._rep_mat = np\
            .array(('',) * items, dtype=object)\
            .reshape(
                (self.height + 2),
                (self.width + 2),
            )

        # Boarder of maze
        self._rep_mat[:, 0] = '|'
        self._rep_mat[:, -1] = '|'
        self._rep_mat[0, :] = '-'
        self._rep_mat[-1, :] = '-'

        if with_path:
            self._rep_mat[1:-1, 1:-1] = self.solved_maze
        else:
            self._rep_mat[1:-1, 1:-1] = self.maze

        if with_open_closed:
            for node in self.open_set:
                self._rep_mat[node] += '*'
            for node in self.closed_set:
                self._rep_mat[node] += 'x'

        for row in range(rows):
            for col in range(cols):
                self._rep_mat[row, col] = self.node_representation(
                    self._rep_mat,
                    (row, col),
                )

        rows, cols = self._rep_mat.shape
        for row in range(rows):
            for col in range(cols):
                print(self._rep_mat[row, col], end='')
            print()

The validity of the code has been confirmed by the tests placed in `mazesolver.tests.test_maze_solver.TestMazeSolver`.

In [ ]:
# %load mazesolver/tests/test_maze_solver.py
import numpy as np
import pytest

from mazesolver import MazeSolver


class TestMazeSolver:
    def test_init_with_file(self):
        with open('mazesolver/tests/boards/board-1-1.txt') as f:
            ms = MazeSolver(f.read())

        assert ms.maze.shape == (7, 20)

    def test_simple_maze_string(self):
        maze = '.#\nAB'
        ms = MazeSolver(maze)

        assert np.array_equal(ms.maze, np.array([['.', '#'], ['A', 'B']]))

    def test_dimensions_of_maze(self):
        maze = '.#.\nAB.'
        ms = MazeSolver(maze)

        assert ms.width == 3
        assert ms.height == 2

    def test_start_and_goal_index(self):
        maze = '.#.\nAB.'
        ms = MazeSolver(maze)

        assert ms.start == (1, 0,)
        assert ms.goal == (1, 1,)

    def test_is_goal_reached(self):
        maze = '.#.\nAB.'
        ms = MazeSolver(maze)

        assert ms.is_goal_reached((0, 0), (0, 0)) is False
        assert ms.is_goal_reached((1, 0), (0, 0)) is False
        assert ms.is_goal_reached((1, 1), (0, 0)) is True

    def test_reachable(self):
        maze = 'A#B\n.#.\n...'
        ms = MazeSolver(maze)

        # Check all points within maze
        assert ms.reachable((0, 0)) is True
        assert ms.reachable((0, 1)) is False
        assert ms.reachable((0, 2)) is True

        assert ms.reachable((1, 0)) is True
        assert ms.reachable((1, 1)) is False
        assert ms.reachable((1, 2)) is True

        assert ms.reachable((2, 0)) is True
        assert ms.reachable((2, 1)) is True
        assert ms.reachable((2, 2)) is True

        # Out of bounds
        assert ms.reachable((-1, 0)) is False
        assert ms.reachable((0, -1)) is False
        assert ms.reachable((3, -1)) is False
        assert ms.reachable((0, 3)) is False

    def test_neighbours(self):
        maze = 'A#B\n.#.\n...'
        ms = MazeSolver(maze)

        assert ms.neighbors((0, 0)) == ((1, 0),)

    def test_complex_neighbours_case(self):
        maze =  '######\n' + \
                '..A.#B\n' + \
                '.####.\n' + \
                '......\n'
        ms = MazeSolver(maze)
        assert ms.neighbors((1, 2)) == (
            (1, 3),
            (1, 1),
        )
        assert ms.neighbors((2, 5)) == (
            (3, 5),
            (1, 5),
        )

    def test_distance_between_adjacent_nodes(self, maze_solver):
        assert maze_solver.distance_between(None, None) == 1

    def test_heuristic_cost_estimate(self, maze_solver):
        f = maze_solver.heuristic_cost_estimate

        assert f((0, 0), (0, 0)) == 0
        assert f((0, 1), (0, 0)) == 1
        assert f((0, 0), (0, 1)) == 1
        assert f((3, 3), (0, 1)) == 5

    def test_solve(self, maze_solver):
        solution = [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (1, 2), (0, 2)]
        assert maze_solver.solve() == solution

    @pytest.mark.skip(reason='Can not find out how to capture stdout')
    def test_visualize(self, maze_solver, capsys):
        maze_solver.visualize()
        out, _ = capsys.readouterr()
        assert out == '-----\n|O#O|\n|O#O|\n|OOO|\n-----\n'

    def test_solving_solvable_mazes(self):
        solvable_maze_numbers = ['1', '2', '3', '4']
        for num in solvable_maze_numbers:
            with open('mazesolver/tests/boards/board-1-' + num + '.txt') as f:
                ms = MazeSolver(f.read())
                ms.solve()
                assert ms.success


## Task 1.2
The visualization of the paths found for the mazes `board-1-{1,2,3,4}` are now shown:

In [ ]:
# %load main.py
from sys import argv

from mazesolver import MazeSolver, TerrainMazeSolver

if argv[1] == '1':
    board_paths = 'mazesolver/tests/boards/board-1-'

    for board_number in [1, 2, 3, 4]:
        with open(board_paths + str(board_number) + '.txt') as board_file:
            maze_solver = MazeSolver(board_file.read())
            maze_solver.visualize()

In [12]:
%run main.py 1

----------------------
|....................|
|....................|
|.........######.....|
|........OOOA..#..B..|
|........O######..O..|
|........OOOOOOOOOO..|
|....................|
----------------------
----------------------
|....OOO#............|
|...OO#O#............|
|..OO#OO#............|
|AOO#.O#....OOOOOOOOB|
|....#OO#..OO#.......|
|.....#OO#OO#........|
|......#OOO#.........|
----------------------
----------------------
|.........OOO........|
|.........O#OO.......|
|.......##OO#OOOO....|
|......#OA#O#...O....|
|......#O#OO#...O....|
|......#OOO#....OO...|


|.......###......OOOB|
----------------------
----------------------
|AO#.......#......#..|
|#O#.#####.#.####.#..|
|OO#OOOOO#.#....#....|
|O##O###O######.#####|
|OO#OB#OO#....#...#..|
|#O####O##.##.#.#.##.|
|.OOOOOO....#...#....|
----------------------


# Task 2
The source code requires minimal alteration in order to support cell costs.

## Task 2.1
The main change in the source code is the addition of a new subclass of `MazeSolver` named `TerrainMazeSolver`, defining a new method for `distance_between(self)`. The source code can be seen below:

In [36]:
# %load mazesolver/__init__.py
from typing import Tuple, List

import colored
from colored import stylize
import numpy as np

from astar import AStar


Node = Tuple[int, int]


class TerrainMazeSolver(MazeSolver):
    REACHABLE_NODES = ['w', 'm', 'f', 'g', 'r', 'A', 'B']
    COST_OF = {
        'w': 100,
        'm': 50,
        'f': 10,
        'g': 5,
        'r': 1,
        # Start and goal nodes need to always be traversed, so the cost does
        # not matter
        'A': 0,
        'B': 0,
    }

    def distance_between(self, n1: Node, n2: Node) -> int:
        return self.COST_OF[self.maze[n2]]

    @classmethod
    def node_representation(cls, maze: np.array, node: Node) -> str:
        STYLE_OF = {
            '-': colored.bg('white'),
            '|': colored.bg('white'),
            'w': colored.bg('blue'),
            'm': colored.bg(243),
            'f': colored.bg(28),
            'g': colored.bg(119),
            'r': colored.bg(130),
            'A': colored.bg(1),
            'B': colored.bg(1),
            'O': colored.bg('dark_orange_3b'),
        }
        type = maze[node]
        return stylize(type[-1], STYLE_OF[type[0]] + colored.fg('white'))


all = ['MazeSolver', 'TerrainMazeSolver']



The validity has, again, been confirmed by tests performed in `mazesolver.tests.test_terrain_maze_solver`:

In [37]:
# %load mazesolver/tests/test_terrain_maze_solver.py
from mazesolver import TerrainMazeSolver


class TestTerrainMazeSolver:
    def test_distance_between_adjacent_nodes(self):
        maze = 'Aw\ngB\n'
        ms = TerrainMazeSolver(maze)

        water_cost = ms.distance_between(
            (0, 0),
            (0, 1),
        )
        assert water_cost == 100

        grasslands_cost = ms.distance_between(
            (1, 0),
            (1, 0),
        )
        assert grasslands_cost == 5

    def test_with_the_given_boards(self):
        solvable_maze_numbers = ['1', '2', '3', '4']

        for num in solvable_maze_numbers:
            with open('mazesolver/tests/boards/board-2-' + num + '.txt') as f:
                ms = TerrainMazeSolver(f.read())
                ms.solve()
                assert ms.success

The visualization of the mazes are done by printing the maze to terminal with ansi esquape sequences. The result is shown below, orange `O` indicating the found path:

In [ ]:
# %load main.py
from sys import argv

from mazesolver import MazeSolver, TerrainMazeSolver

if argv[1] == '2':
    board_paths = 'mazesolver/tests/boards/board-2-'

    for board_number in [1, 2, 3, 4]:
        with open(board_paths + str(board_number) + '.txt') as board_file:
            maze_solver = TerrainMazeSolver(board_file.read())
   

In [10]:
% run main.py 2

------------------------------------------
|mmmmmffffrrrrrrrrArrrrrrrrrrrrrrfffmmmmm|
|mmmffffffffrrrrrrOOOOOOOOOOOOOrfffffmmmm|
|mmfffffffffffffffffffffffffffOffffffmmmm|
|mmfffffffffffffwwwwwfffffffffOfffffffmmm|
|mfffffffffffffwwwwwwwffffffffOffffffmmmm|
|mmffffffffffffwwwwwwwffOOOOOOOrrrrrrmmmm|
|mmmffffffffffffwwwwwfffOffffffffrffffmmm|
|mmfffffffffffffffffffffOffffffffrfffffmm|
|mmffffffffgggggggOOOOOOOggggggggggffffmm|
|mmmffffggggggggggBggggggggggggggggggffmm|
------------------------------------------
------------------------------------------
|ffffffffffgggrgggggggrggggfffffffrrfffff|
|ffAffffffgggrrggggggrrggffffffffrrffffff|
|ffOfffgggggrrggggggrrgggffffrrrrrfffffff|
|ggOggggggggrggggrrrrgggffffrrfffffffffff|
|ggOOOOOOOOOOrrrrrgggggffffrrffffffffffff|
|ggggrrgggggOggggggggffffffrfffffffffffff|
|gggrrggggggOOggggggffffOOOOOOfffffffffff|
|ggrrgggffgggOOOOggffOOOOfrffOrrrrfffffff|
|ggrggffffffffffOOOOOOffffrffOfffrrffffff|


|ggrgfffffffffffffffffffffrffBffffrrfffff|
------------------------------------------
------------------------------------------
|gggggggggwwwgggggmmmmmmmmmmBOOOOOOOmmmmm|
|gggggggggwwwwggggmmmmmmmmmmmmmmmmmOmgggg|
|ggggggggggwwwwggggmmmmmmmmmmmmmmggOggggg|
|ffgggggggggwwwwggggmmmOOOOmmOOOOOOOrgggg|
|ffggggggggggwwwwwwwwwwOwwOOOOggggggrrrrr|
|ffffggggggggggwwwwwwwwOwwwwggggggggggggg|
|fffffOOOOOOOOOOOOOOOOOOwwwwwwggggggmmmmm|
|fAOOOOfffffgggggggggmmmmmmwwwwmmmmmmmmmm|
|fffffffffffffffmmmmmmmmmmmmmwwwmmmmmmmmm|
|ffffffffffmmmmmmmmmmmmmmmmmmwwwmmmmmmmmm|
------------------------------------------
------------------------------------------
|wwwwwgggggggggggggggggggOOOOOOOOOOgrrrrr|
|wwwwwwwgggggggggggggOOOOOgggwwwwwOOrgggg|
|wwwwwwwwwwwgggAOOOOOOgwwwwwwwwwwwwOwgggg|


|wwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwOwwwww|
|wwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwOwwwww|
|wwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwOOwwwww|
|wwwwwwwwgggggBgggggwwwwwwwwwwwwwwOwwwwww|
|wwwwggggggfffOffgggggggggwwwwwwwwOwwwwww|
|wwggggfffffffOffffffggggggggggggOOgwwwww|
|wgggfffrrrrrrOOOOOOOOOOOOOOOOOOOOggggggg|
------------------------------------------


# Task 3
## Task 3.1
The source code for the Astar method has been altered in order to allow the use of `BFS` or `Dijkstra` instead. The changes made to the original code can be seen in [this commit](https://github.com/JakobGM/Astar/commit/876058e4b4316c431ec9b50c7f7a322d395693cb). To summarize:
* The queue type is chosen according the method.
* The open and closed nodes are saved for further inspection.

## Task 3.2
`main.py` now takes in an argument `3` which shows the solution found the respective three methods for boards `1-1` and `1-2`. Open nodes are shown as `*` and closed ones as `x`.

In [15]:
# %load main.py
from sys import argv

if argv[1] == '3':
    base_path = 'mazesolver/tests/boards/'
    board_names = (
        ('board-1-1.txt', MazeSolver),
        ('board-2-1.txt', TerrainMazeSolver),
    )
    methods = ('Astar', 'Dijkstra', 'BFS',)

    for board_name, Solver in board_names:
        for method in methods:
            with open(base_path + board_name) as board_file:
                maze_solver = Solver(board_file.read())
                maze_solver.solve(method=method)

                print('\n' + '\u2500' * 60)
                print(
                    'Solution for board "{}" found with the {}-algorithm'\
                    .format(board_name, method)
                )
                # maze_solver.visualize()
                maze_solver.representation(with_open_closed=True)
                print('\u2500' * 60)


In [8]:
%run main.py 3


────────────────────────────────────────────────────────────
Solution for board "board-1-1.txt" found with the Astar-algorithm
--------*******-------
|......*xxxxxxx*.....|
|.....*xx......**....|
|....*xxxxxxxx#xx....|
|.....*xxOOOA..xxx*..|
|......*xxxxxxxxxx*..|
|.......**********O..|
|....................|
----------------------
────────────────────────────────────────────────────────────

────────────────────────────────────────────────────────────
Solution for board "board-1-1.txt" found with the Dijkstra-algorithm
xxxxxxxxxxxxxxxxxx*---
xxxxxxxxxxxxxxxxxxx*.|
xxxxxxxxx......xxxx*.|
xxxxxxxxxxxxxx#xx.*..|
xxxxxxxxxOOOA..xxx*..|
xxxxxxxxxxxxxxxxxxx*.|
xxxxxxxxxxxxxxxxxx*..|
|....................|
----------------------
────────────────────────────────────────────────────────────

────────────────────────────────────────────────────────────
Solution for board "board-1-1.txt" found with the BFS-algorithm
xxxxxxxxxxxxxxxxxx*---


xxxxxxxxxxxxxxxxxxx*.|
xxxxxxxxx......xxx*..|
xxxxxxxxxxxxxx#xx.*..|
xxxxxxxxxOOOA..xxxx*.|
xxxxxxxxxxxxxxxxxxxx.|
xxxxxxxxxxxxxxxxxxx*.|
|....................|
----------------------
────────────────────────────────────────────────────────────

────────────────────────────────────────────────────────────
Solution for board "board-2-1.txt" found with the Astar-algorithm
----*xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx*------
|mmm*xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx*mmmmm|
|mmmf*xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx*mmmm|
|mmfff*xxxxxxxx*****xxxxxxxxxxxxxxxxx*mmm|
|mmffff*xxxxxx*fwwww*xxxxxxxxxxxxxxx*fmmm|
|mffffff*xxxxx*wwwww*xxxxxxxxxxxxxxx*mmmm|
|mmffffff*xxxxx*www*xxxxxxxxxxxxxxxxx*mmm|


|mmmffffff*xxx*fw***xxxxxxxxxxxxxxxxx*mmm|
|mmfffffff*xxxx**xxxxxxxxxxxxxxxxxxx*ffmm|
|mmffffffff****ggg*xxxxxxxxxxxxxxxxxx*fmm|
|mmmffffggggggggggBggggggggggggggggggffmm|
------------------------------------------
────────────────────────────────────────────────────────────

────────────────────────────────────────────────────────────
Solution for board "board-2-1.txt" found with the Dijkstra-algorithm
---**xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx*------
|m*xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx*mmmm|
|mm*xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx*mmmm|
|mmf*xxxxxxxxxx*****xxxxxxxxxxxxxxxxx*mmm|
|mmff*xxxxxxxx*fwwww*xxxxxxxxxxxxxxx*fmmm|
|mffff*xxxxxxx*wwwww*xxxxxxxxxxxxxxxx*mmm|
|mmffff*xxxxxxx*www*xxxxxxxxxxxxxxxxx*mmm|
|mmmffff*xxxxxx*w**xxxxxxxxxxxxxxxxxxx*mm|
|mmffffff*xxxxx**xxxxxxxxxxxxxxxxxxxxx*mm|
|mmfffffff*xxx*gggxxxxxxxxxxxxxxxxxxxx*mm|
|mmmffffggggggggggBOOggggggggggggggggffmm|
------------------------------------------
────────────────────────────────────────────────────────────

───────────────

## Task 3.3
a)
We see that the BFS algorithm does not take the cost of the route into account at all. For the board `2-1` the straight route across the water is chosen, as this is the first valid solution found.

Dijkstra and A* finds equally cheap solutions to these mazes, but A* chooses the path where the nodes of the path are on average closer to the end goal, even if it ends up with the same cost in the end.

b)
Although Dijkstra and A* chooses the same path, Dijkstra ends up having to close more nodes, thus being less efficient. BFS is less predictable, it can close more or less nodes, but it finds worse solutons to the mazes.